# Title NN classifier for Political Bia

In [1]:
import pandas as pd
import numpy as np

batch_size = 64

In [2]:
df = pd.read_csv("../data/source/newsclust.csv")
df = df.query("site != 'cbn.com'")
print(len(df))
df = df.sample(90000)

103262


Compute the bias for each one of the articles, based on the publication's known bias

In [3]:
from bias import Bias

num_classes = 7
df['bias'] = df.apply(lambda row: Bias.get_bias_for_domain(row['site']).value, axis=1)
df.head(2)

,Unnamed: 0,date,site,text,title,url,bias
3607,3607,2016-03-07T13:19:00.000+02:00,ft.com,Iran’s hardliners down but not out after poll ...,Iran’s hardliners down but not out after poll ...,http://www.ft.com/cms/s/0/3c499716-e055-11e5-9...,3
39744,39744,2017-05-13T06:11:00.000+03:00,oppositionreport.com,Democrats Discuss Path To Trump Impeachment — ...,Democrats Discuss Path To Trump Impeachment — ...,http://oppositionreport.com/democrats-discuss-...,0


Tokenize the text of the articles, create a vocabulary of words

In [4]:
import spacy
nlp = spacy.load('en')
nlp.pipeline = []

def tokenize_text(texts):
    docs = [doc for doc in nlp.pipe(texts, batch_size=500, n_threads=8)]
    return docs

def is_invalid_token(token):
    return token.is_punct or token.is_space or token.like_url or token.like_num or token.is_digit

def get_words_for_docs(docs):
    return [get_words_ids(doc) for doc in docs]

def get_words_ids(doc):
    return [token.orth for token in doc if not is_invalid_token(token)]

In [5]:
df['docs'] = tokenize_text(df['title'].astype(str))

In [6]:
df['words'] = get_words_for_docs(df['docs'])
df.head(1)

,Unnamed: 0,date,site,text,title,url,bias,docs,words
3607,3607,2016-03-07T13:19:00.000+02:00,ft.com,Iran’s hardliners down but not out after poll ...,Iran’s hardliners down but not out after poll ...,http://www.ft.com/cms/s/0/3c499716-e055-11e5-9...,3,"(Iran, ’s, hardliners, down, but, not, out, af...","[2385, 1402, 295995, 765, 559, 538, 582, 805, ..."


Keep the most common words as vocabulary, replace the words out of vocabulary by the least frequently used word

In [7]:
from collections import Counter

vocab_size = 5000

def flatten(l):
    flat_list = [item for sublist in l for item in sublist]
    return flat_list

word_freq = Counter(flatten(df['words']))
common_words = word_freq.most_common(vocab_size)
word_freq.most_common(5)

[(504, 18205), (576246, 14814), (522, 12694), (501, 11626), (566, 11506)]

In [8]:
print(nlp.vocab.strings[501], nlp.vocab.strings[510], nlp.vocab.strings[512], nlp.vocab.strings[506])

the of and a


In [9]:
vocab = {word_id: i for (i, (word_id, freq)) in enumerate(common_words)}
print(len(vocab), vocab[501], vocab[504], vocab[510])
oov_word = vocab_size - 1
oov_word

5000 3 0 5


4999

In [10]:
def get_words_ids_if_common(words):
    return [vocab.get(word, oov_word) for word in words]

df['words_with_oov'] = df.apply(lambda row: get_words_ids_if_common(row['words']), axis=1)
df['words_with_oov'].head()

3607     [222, 6, 4999, 249, 136, 110, 112, 57, 1018, 3...
39744    [141, 4065, 4108, 13, 1, 2773, 955, 3300, 364,...
72172    [19, 941, 50, 4999, 4999, 2306, 20, 21, 3138, ...
29279    [99, 3, 357, 2, 92, 2232, 86, 9, 11, 1899, 0, ...
91596    [4999, 4999, 4999, 4999, 3056, 1581, 276, 4319...
Name: words_with_oov, dtype: object

Look at the distribution of words. The longest article has 15K words, the shortest 21.

In [11]:
lens = np.array(list(map(len, df['words'])))
(lens.max(), lens.min(), lens.mean())

(45, 0, 11.102155555555555)

Split the data for training and validation

In [12]:
TEST_DOMAINS =  {
    'bloomberg.com',
     'breitbart.com',
     'c-span.org',
     'chicagotribune.com',
     'chron.com'
}
df_train =  df.loc[~df['site'].isin(TEST_DOMAINS)]
df_test =  df.loc[df['site'].isin(TEST_DOMAINS)]
print(len(df_train))
print(len(df_test))

82284
7716


Enforce all texts to have the same length, truncating or padding if necessary

In [13]:
seq_len = 50

from keras.preprocessing import sequence

trn = sequence.pad_sequences(df_train['words_with_oov'], maxlen=seq_len, value=0)
test = sequence.pad_sequences(df_test['words_with_oov'], maxlen=seq_len, value=0)

trn

Using TensorFlow backend.


array([[   0,    0,    0, ...,   57, 1018, 3518],
       [   0,    0,    0, ..., 3300,  364, 1453],
       [   0,    0,    0, ...,   97,  516,  299],
       ..., 
       [   0,    0,    0, ...,    7,  870, 1830],
       [   0,    0,    0, ...,  101,    4, 4999],
       [   0,    0,    0, ...,    5, 4999,  255]], dtype=int32)

## Single Hidden Layer NN classifier

### Keras NN model

In [14]:
# Force use CPU
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import keras
from keras.models import Sequential, Model
from keras.layers import Embedding
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import Adam

In [15]:
k_model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(7, activation='softmax')])

In [16]:
k_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
k_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 50, 32)        160000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 1600)          0           embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 100)           160100      flatten_1[0][0]                  
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 100)           0           dense_1[0][0]                    
___________________________________________________________________________________________

In [17]:
from keras.utils.np_utils import to_categorical

labels_train = np.array(df_train['bias'])
labels_test = np.array(df_test['bias'])

print(labels_test[:3])
k_model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=3, batch_size=batch_size)

[2 5 2]
Train on 82284 samples, validate on 7716 samples
Epoch 1/3
82284/82284 [==============================] - 7s - loss: 1.3418 - acc: 0.4551 - val_loss: 1.1584 - val_acc: 0.6085
Epoch 2/3
82284/82284 [==============================] - 7s - loss: 1.0179 - acc: 0.6025 - val_loss: 1.0820 - val_acc: 0.5989
Epoch 3/3
82284/82284 [==============================] - 8s - loss: 0.9033 - acc: 0.6544 - val_loss: 1.1220 - val_acc: 0.5779


Manually verify some articles. The bias for these articles should match the predicted bias below.

In [18]:
df_sample = df_test.sample(5)
df_sample[['site', 'bias']]

,site,bias
71514,chicagotribune.com,2
59478,breitbart.com,5
497,chicagotribune.com,2
34585,c-span.org,3
36709,breitbart.com,5


In [19]:
sample = sequence.pad_sequences(df_sample['words_with_oov'], maxlen=seq_len, value=0)
print("precision = ", np.sum(np.argmax(k_model.predict(test), axis=1) == labels_test) * 100.0 / len(labels_test))
np.argmax(k_model.predict(sample), axis=0)

precision =  57.7890098497


array([1, 4, 0, 3, 0, 1, 1])

## Site vectors

Remove the last linear classifier

In [20]:
l4 = k_model.pop()
l3 = k_model.pop()

In [21]:
k_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
k_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 50, 32)        160000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 1600)          0           embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 100)           160100      flatten_1[0][0]                  
Total params: 320,100
Trainable params: 320,100
Non-trainable params: 0
____________________________________________________________________________________________________


In [22]:
article_vectors = k_model.predict(trn)
article_vectors.shape

(82284, 100)

Generate an average vector of all the article vectors for each site

In [23]:
site_vectors = []
labels = []
sample_size = 1000
for site in df_train['site'].unique():
    site_indexes = np.where(df_train["site"] == site)[0]
    if len(site_indexes) > 0:
        site_vector = np.mean(article_vectors[site_indexes[:sample_size], :], axis=0)
        site_vectors.append(site_vector)
        labels.append(site)
site_vectors = np.array(site_vectors)
site_vectors.shape

(57, 100)

In [24]:
from sklearn.manifold import TSNE

tsne_model = TSNE(n_components=2, random_state=0)
# np.set_printoptions(suppress=True)
site_vectors_2_dim = tsne_model.fit_transform(site_vectors)
X_proj = site_vectors_2_dim[:, 0]
Y_proj = site_vectors_2_dim[:, 1]

In [25]:
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from plotly.graph_objs import Bar, Scatter, Figure, Layout, XAxis, YAxis

In [26]:
trace = Scatter(x=X_proj, y=Y_proj, mode='markers+text', text=labels, textposition='top',  marker=dict(size=10))
iplot({
    'data': [trace],
    'layout': Layout(
        #xaxis=XAxis(title='Left vs Right'), 
        #yaxis=YAxis(title='Biased vs Factual'),
        autosize=False,
        width=1000,
        height=700)},
    show_link=False
)

### PCA Analysis

In [27]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
site_vectors_pca = pca.fit(site_vectors).components_

In [28]:
trace = Scatter(x=site_vectors_pca[0], y=site_vectors_pca[1], mode='markers+text', text=labels, textposition='top',  marker=dict(size=10))
iplot({
    'data': [trace],
    'layout': Layout(
        #xaxis=XAxis(title='Left vs Right'), 
        #yaxis=YAxis(title='Biased vs Factual'),
        autosize=False,
        width=1000,
        height=700)},
    show_link=False
)